CRNN Base Structure (TODO Training and Accuracy)

In [5]:
# Import required libraries
import numpy as np
import torch
import torch.nn as nn

In [6]:
# Base structure of CRNN
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN, self).__init__()
        self.features = nn.Sequential (
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3,3)),
            nn.BatchNorm2d(64),
            nn.ELU(),
            nn.MaxPool2d((2,2)),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3)),
            nn.BatchNorm2d(128),
            nn.ELU(),
            nn.MaxPool2d((2, 2)),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3)),
            nn.BatchNorm2d(256),
            nn.ELU(),
            nn.MaxPool2d((2, 2)),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3)),
            nn.BatchNorm2d(256),
            nn.ELU(),
            nn.MaxPool2d((2, 2)),

            nn.Flatten()
        )

        W = 86 // (2**4)
        H = 96 // (2**4)
        C = 256

        totalInstruments = 3

        self.gru = nn.GRU(
            input_size=C*H, 
            hidden_size=256, 
            batch_first=True
        )
        # out_features is technically 18 in paper, but realistically 3 at the beginning
        self.fc = nn.Linear(in_features=256, out_features=totalInstruments)
        
    def forward(self, input):
        conv_output = self.features(input)
        gru_output, _ = self.gru(conv_output)

        fc_input = gru_output[:, -1, :]

        return self.fc(fc_input)

In [7]:
# Future stuff for training and accuracy of CNN
import torch.optim as optim

model = CRNN()
lossAlg = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0001, momentum=0.9)

In [25]:
# Training the model
from torch.utils.data import DataLoader
from utils import *

batch_size = 32
#dataloader = DataLoader(dataset, batch_size = batch_size, shuffle=True)

num_epochs = 20
for epoch in range(num_epochs):
    #for batch in dataloader:
        #input, labels = batch
        #output = CRNN(input)
        #loss = lossAlg(output.view(-1), labels.view(-1))
        #optimizer.zero_grad()
        
        # optimizer.step()
    #print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
    break